In [118]:
!pip install boto3 pandas scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [119]:
import boto3
import json
import pandas as pd
from sklearn.metrics import classification_report

# Initialize a session using Amazon Bedrock
client = boto3.client('bedrock-runtime')

In [120]:
data = {
    'Query': [
        'Why is my bill higher this month?',
        'How do I reset my password?',
        'Can I get a discount on my next purchase?',
        'What are your business hours?',
        'I need help with my internet connection',
        'Do you offer any promotions?',
        'When will my order be delivered?',
        'How can I update my payment method?',
        'Is there a warranty on this product?',
        'Can I speak to a human representative?'
    ],
    'Intent': [
        'Billing Inquiry',
        'Technical Support',
        'Sales Inquiry',
        'General Information',
        'Technical Support',
        'Sales Inquiry',
        'Order Status',
        'Account Management',
        'Product Information',
        'Customer Service'
    ]
}

df = pd.DataFrame(data)
print(df)

                                       Query               Intent
0          Why is my bill higher this month?      Billing Inquiry
1                How do I reset my password?    Technical Support
2  Can I get a discount on my next purchase?        Sales Inquiry
3              What are your business hours?  General Information
4    I need help with my internet connection    Technical Support
5               Do you offer any promotions?        Sales Inquiry
6           When will my order be delivered?         Order Status
7        How can I update my payment method?   Account Management
8       Is there a warranty on this product?  Product Information
9     Can I speak to a human representative?     Customer Service


In [121]:
def classify_intent(query):
    prompt = f"""
    You are an AI assistant trained to classify customer support queries into intent categories.
    Given the following query, classify it into one of these intent categories:
    - Billing Inquiry
    - Technical Support
    - Sales Inquiry
    - General Information
    - Order Status
    - Account Management
    - Product Information
    - Customer Service

    Query: {query}

    Respond with only the intent category, nothing else.
    """

    payload = {
        "prompt": f"\n\nHuman: {prompt}\n\nAssistant:",
        "max_tokens_to_sample": 100,
        "temperature": 0.1,
        "top_p": 1
    }

    response = client.invoke_model(
        modelId='anthropic.claude-v2', 
        contentType='application/json',
        accept='application/json',
        body=json.dumps(payload)
    )

    response_body = json.loads(response['body'].read())
    return response_body['completion'].strip()

In [122]:
y_true = df['Intent'].tolist()
y_pred = []

for query in df['Query']:
    intent = classify_intent(query)
    y_pred.append(intent)
    print(f"Query: {query}")
    print(f"Predicted Intent: {intent}")
    print("---")

# Generate classification report
report = classification_report(y_true, y_pred)
print("\nClassification Report:")
print(report)

Query: Why is my bill higher this month?
Predicted Intent: Billing Inquiry
---
Query: How do I reset my password?
Predicted Intent: Technical Support
---
Query: Can I get a discount on my next purchase?
Predicted Intent: Sales Inquiry
---
Query: What are your business hours?
Predicted Intent: General Information
---
Query: I need help with my internet connection
Predicted Intent: Technical Support
---
Query: Do you offer any promotions?
Predicted Intent: Sales Inquiry
---
Query: When will my order be delivered?
Predicted Intent: Order Status
---
Query: How can I update my payment method?
Predicted Intent: Billing Inquiry
---
Query: Is there a warranty on this product?
Predicted Intent: Product Information
---
Query: Can I speak to a human representative?
Predicted Intent: General Information
---

Classification Report:
                     precision    recall  f1-score   support

 Account Management       0.00      0.00      0.00         1
    Billing Inquiry       0.50      1.00      

/Users/leetan/.local/share/virtualenvs/gpt-engineer-7XFB_0JR/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/leetan/.local/share/virtualenvs/gpt-engineer-7XFB_0JR/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/leetan/.local/share/virtualenvs/gpt-engineer-7XFB_0JR/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param